# Notebook to experiment with MongoDB

In this notebook you can play around with MongoDB through python.

First we install some needed libraries.

In [ ]:
%pip install pymongo --quiet
%pip install folium --quiet

Next we load the libraries we are going to use. Most important is pymongo which will interface python with the MongoDB running on the Jupyter server.

In [ ]:
import os
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import pymongo
from pymongo import MongoClient
import subprocess

Print the version of MongoDB.

In [ ]:
print ('Mongo version', pymongo.__version__)


Connect to the MongoDB on localhost using user `test`. The password is grabbed from the disk, to hide it from others.
We are connecting to the database test. The variable `client` holds the connection from the client to the server.

In [ ]:
host = "localhost"
port = 27017

user_name = "test"
pass_word = subprocess.check_output(['cat', '/home/mongopasswd']).decode("utf-8").replace("\n", "")  

db_name = "test"  # database name to authenticate

client = MongoClient(f'mongodb://{user_name}:{pass_word}@{host}:{port}/{db_name}') 


The `db` variable holds the database.
The `collection` variable holds the collection (equivalent to a table in SQL) we are going to be working with.

**Change the text FILL_IN to something of your choice - different from the other participants**

In [ ]:
db = client.test
collection = db.FILL_IN

If there is already something in the collection we drop it.
Then we import the JSON data from the dummyData.json file.

**FILL_IN in the mongoimport command needs to be the same collection name as you picked in the previous code block**

In [ ]:
collection.drop()
os.system('mongoimport -u test -p "mongo" -d test -c FILL_IN ./dummyData.json')

In the following code block we perform a query. We find everything in the collection, sort Ascending on Age and limit the output to 10 entries.
The result is a cursor - this is an iterable in python. So we loop over cursor to get all the rows of output.

In [ ]:
cursor = collection.find().sort('Age',pymongo.ASCENDING).limit(10)
for doc in cursor:
    print (doc)

In the following code block we create an aggregate pipeline.
In the pipeline we state we want to group over Country. For each Country we calculate the average age, called AvgAge.
We also count how many entries per Country.

We sort the resulting rows ascending on AvgAge and descending on count.

The result is again a cursor, this time we use list to convert the iterable into an array. Which we wrap in a Pandas data-frame.

We show the first 10 entries in data-frame df1.

In [ ]:
pipeline = [
        {"$group": {"_id":"$Country",
             "AvgAge":{"$avg":"$Age"},
             "Count":{"$sum":1},
        }},
        {"$sort":{"Count":-1,"AvgAge":1}}
]
aggResult = collection.aggregate(pipeline) # returns a cursor
df1 = pd.DataFrame(list(aggResult)) # use list to turn the cursor to an array of documents
df1 = df1.set_index("_id")
df1.head(10)

This next query does a match against the Country being China. So all 'rows' that contain China are returned.
Again use list and pd.DataFrame to convert to a Pandas data-frame.

Show the first 10 entries.

In [ ]:
pipeline = [
        {"$match": {"Country":"China"}},
]
aggResult = collection.aggregate(pipeline)
df2 = pd.DataFrame(list(aggResult))
df2.head(10)

In the next code block we use `folium` to show the results in the `df2` data-frame on a map.

In [ ]:
import folium
print ('Folium version', folium.__version__)

world_map = folium.Map(location=[35, 100], 
                    zoom_start=4)
for i in range(len(df2)):
    folium.Marker(df2.Location[i].split(','), popup=df2.Name[i]+', age:'+str(df2.Age[i]),tooltip=df2.Name[i]).add_to(world_map)
world_map